# Л2: Использование Inference API для работы с моделями ИИ

**Inference API** - это сервис, который обеспечивает быстрый доступ к многочисленным предобученным моделями ИИ, размещенным на инфраструктуре HuggingFace. Используя его нет необходимости в загрузке модели и запуску модели локально. Таким образом можно создавать прототипы приложений для решения различных задач не переживая о вычислительных мощностях своего компьютера. 

**Inference API** предоставляет бесплатный и мгновенный доступ к популярным и эффективным моделями для решения следующего спектра задача:
* Генерация текста: включает большие языковые модели и подсказки для вызова инструментов, генерируйте и экспериментируйте с высококачественными ответами.
* Генерация изображений: легко создавайте персонализированные изображения.
* Работа с документами.
* Классические задачи ИИ: готовые к использованию модели для классификации текста, изображений, распознавания речи и многого другого.


**В этой работе мы познакомимся с этим инструментом и в качестве примера создадим собственного ассистента.**

## 1/ Подготовка среды выполнения
На данной этапе вам наобходимо подготовить виртуальное окружение и установить все необходимые библиотеки.

1. Создать и активировать (или только активировать, если ранне создавали) виртуальной окружение `python`.

В терминале вводим следующие команды команды:

*Создаем виртуальное окружение с помощью `python-venv`*
```
python -m venv env
```
*Активируем виртуальное окружение*
для CMD
```
env\Scripts\activate
```
для PowerShell
```
env\Scripts\Activate.ps1
```
**Примечание.** `env` - это название вашего виртуального окружения, назвать его можете как угодно.

После этого можем выбрать наш локальный интерпрететор pyhton, нажав на кнопку выше "Select kernel".

2. Устанавливаем все необходимые библиотеки

**Примечание.** Библиотеки установятся внутрь вашего виртуального окружения.

Нам понадобятся библиотеки Diffusers, Transformers, Accelerate.

```
pip install transformers
```
Также для работы вышеперечисленных бибилиотек потребуется PyTorch:
```
pip install torch
```
Библиотека Gradio для создания web-приложения.
```
pip install gradio
```
Также для работы с изображение нам потребуется библиотека PIL (Python Image Library) `Pillow`
```
pip install Pillow
```

## 2/ Начало рабты. Создание Inference Client

Для бессерверного обращение к модели (то есть без запуска ее на своем каком-то сервере или локально) необходимо отправить запрос, используя Inference API. В результате этого запроса мы получим ответ - Inference (то есть вывод модели). Запрос можно формировать разынми способами, но в этой работе предлагается использовать Python библиотеку `huggingface_hub`. Выполните установку
```
pip install --upgrade huggingface_hub
```

Эта библиотека предоставляет модуль `InferenceClient`, с помощью которого создадим клиент, который будет формировать запрос на сервера Hugging Face. Импортируем этот модуль. Также импортируете Gradio.

In [ ]:
from huggingface_hub import InferenceClient
import gradio as gr

Есть одно НО! Для использования Serverless Inference API необходимо иметь **токен доступа**. Чтобы его получить необходимо зарегистрироваться на сайте Hugging Face, затем перейдите на [странице создания токенов](https://huggingface.co/settings/tokens/new?globalPermissions=inference.serverless.write&tokenType=fineGrained). Создайте `fine-grained` токен с областью действия `Make calls to the serverless Inference API`.

***Примечание***. Не распространяйте свой токен в публичных местах, иначе он будет скопрометирован и Hugging Face его удалит! Токен можно скопировать только один раз в моменте его создания. Сохраните его где-нибудь лично у себя и используете только в коде своего приложения когда необходимо.

Вставьте сюда сгенерированный токен.

In [ ]:
hf_token = "hf_********************"

Для создания умного ассистента, с который способен генерировать связанный текст и поддерживать диалог будем использовать модель генерации текста `openai/gpt-oss-20b`. Это недавно выпущенная открытая LLM от OpenAI.

In [ ]:
ai_model = "openai/gpt-oss-20b"

Создадим клиента через класс `InferenceClient`, указав ему используемую модель и свой токен доступа:

In [ ]:
client = InferenceClient(model=ai_model, token=hf_token)

## 3/ Тестирование работы модели в режиме чата (`chat completion`)
Теперь используя объект `client` мы можем выполнять различны задачи. Например, мы можем встпуить в чат с модель, передав ее следюущий промпт:

In [ ]:
msgs = [{"role": "user", "content": "What is the capital of Great Britan?"}]

Это список всех сообщений, то есть история чата. Пока в нем только одно наше сообщение. Формат сообщения такой, что каждое сообщение это словарь, где указываем роль `"role"`, а по ключу `"content"` находится само сообщение.

Распределение ролей:
* `"user"` - это мы
* моодель отвечает под ролью `"assistant"`

Передаем через клиента наш запрос. Также указываем максимальный размер ответного соощения `max_tokens`

In [ ]:
ans = client.chat_completion(messages=msgs, max_tokens=100, stream=False)

Выведем, что нам ответила модель ИИ. В ответ мы получаем json формат. Вот так вытаскивем само сообщение:

In [ ]:
print(ans.choices[0].message.content)

Вот [здесь](https://huggingface.co/docs/api-inference/tasks/chat-completion) можно почитать подробнее про организацию чата с ИИ и структуру ответов

Для дальнейшей работы необходимо сохранять историю переписки. Поэтому все сообщения (наши и модели) складываем в `msgs` по порядку. Обязательно указываем роли: Роль и контент сообщения от ИИ мы можем вытащить из структуры ответа `ans`

In [ ]:
msgs.append({"role": ans.choices[0].message.role, "content": ans.choices[0].message.content})

Вот получается такая история переписки:

In [ ]:
msgs

Теперь отправим наше следующее сообщение, чтобы поддержвать разговор. Поинтересуемся, сколько человек проживает в столице. Впишем в наш диалог `msgs` наше сообщения в соответствии с формой:

In [ ]:
msgs.append({"role": "user", "content": "How many peoples lives in the capital city?"})

In [ ]:
msgs

Продолжаем разговор:

In [ ]:
ans = client.chat_completion(messages=msgs, max_tokens=100, stream=False)
print(ans.choices[0].message.content)

И также сохраняем ответ в историю переписки

In [ ]:
msgs.append({"role": ans.choices[0].message.role, "content": ans.choices[0].message.content})

### Задание 1 ✅

Проведите небольшой диалог в таком формате с моделью ИИ. Попробуйте поговорить с ней на русском языке. 

Попробуйте использовать другую открытую модель, например от Google `google/gemma-3-270m` (https://huggingface.co/google/gemma-3-270m). Для этого при создании клиента укажите новую модель. Вы также можете выбрать любую другую доступную открытую LLM на https://huggingface.co/models, отсортировав модели по задаче "Text Generation".

## 4/ Создание WEB-интерфейса с помощью `Gradio`

Создайте приложение Gradio в формате чата. В Gradio есть готовый блок `gr.ChatInterface`. Ему необходимо указать функцию обработки сообщений, в которую вы поместите обращение клиента к модели ИИ как это было выше. Функция принимает на вход два параметра
* `message` - текущее сообщение, текст
* `history` - история сообщений с чат-ботом, список из словарей, с указанием ролей

Например, пусть это будет функция с названием chatbot
```python
def chatbot(message, history):
    ...
```


Внутри функции необходимо сформировать историю сообщения с учетом текущего сообщения, то есть добавить в конце `message`. `message` - это просто текст вашего сообщения, его необходимо правильно внести в историю сообщений, то есть указать роль, что сообщение от вашего имени `"user"`. Смотрим как это было реализовано выше и делаем также:

```python
history.append({"role": "user", "content": message})
```

Затем добавляем строчку запросу к моделии ИИ
```python
ans = client.chat_completion(messages=history, max_tokens=1000, stream=False)
```

Увеличим количество токенов в ответе, чтобы получать развернутые сообщения.

В конце мы должны вернуть текст ответного сообщения, полученного от модели ИИ
```python
return ans.choices[0].message.content
```

В итоге должны получить вот такую функцию:

In [ ]:
def chatbot(message, history):
    history.append({"role": "user", "content": message})
    ans = client.chat_completion(messages=history, max_tokens=1000, stream=False)
    return ans.choices[0].message.content

Запуск интферейса Gradio `gr.ChatInterface`. Указываем функцию, тип сообщений и заголовк блока

In [ ]:
demo = gr.ChatInterface(fn=chatbot, type="messages", title="Chat with LLM")
demo.launch()

### ЗАДАНИЕ 2 ✅
На основе примеров кода выше написать готовое Python приложение в отдельном скрипте. Обратите внимание на параметр `max_tokens`, если вы видите, что ответ модели обрезан, необходимо увеличить значения этого параметра.

Дополнительно. При работе чата вы увидите, что ваше введеное сообщение в интерфейсе чата повторяется дважды. Это возникает из-за ошибки формирования истории сообщения в функции `chatbot()`. Вам необходимо исправить эту ошибку, для этого проанализируйте содержимое параметра `history`. Вероятно его необходимо конвертировать в **правильный формат истории сообщения** (см. выше)

## Форма отчетности
В качестве отчета по лабораторной работе вам необходимо предоставить:
1. Файл с кодом вашего приложения
2. Скриншот работы графического web-интерфейса вашего приложения